# Загрузка данных и библиотек

In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_files
import srt
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from datasets import Dataset,DatasetDict
from transformers import TFAutoModelForSequenceClassification,AutoTokenizer, TFTrainingArguments, TFTrainer
import tensorflow as tf
#nltk.download('stopwords')
#nltk.download('wordnet')

sns.set()
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

RuntimeError: Failed to import transformers.training_args_tf because of the following error (look up to see its traceback):
module 'tensorflow.python.util.dispatch' has no attribute 'add_fallback_dispatch_list'

In [ ]:
movie_labels = pd.read_excel('movies_labels.xlsx', names=['id', 'movie', 'level']).drop('id', axis=1)
subtitles_files = load_files('Subtitles/', shuffle=False)

In [ ]:
print(movie_labels.info())
movie_labels.head()

In [ ]:
subtitles = pd.DataFrame(data={'data': subtitles_files.data, 
                          'filenames': subtitles_files.filenames})
subtitles.head()

# Предобработка данных

## subtitles

deleted :
* Crown, The S01E03 - Windsor.en.FORCED.srt
* Crown, The S01E05 - Smoke and Mirrors.en.FORCED.srt
* Crown, The S01E07 - Scientia Potentia Est.en.FORCED.srt
* Crown, The S01E10 - Gloriana.en.FORCED.srt

reformatted:
* Suits season 4

In [ ]:
subtitles.head()

In [ ]:
subtitles['data'] = subtitles['data'].str.decode('ISO-8859-1')

In [ ]:
def get_name_and_level(row):
    level = row.split('\\')[0][10:]
    name = row.split('\\')[1][:-4]
    return pd.Series([name, level])

In [ ]:
subtitles[['movie', 'level']] = subtitles['filenames'].apply(get_name_and_level)
subtitles = subtitles.drop('filenames', axis=1)

In [ ]:
subtitles['movie'] = subtitles['movie'].str.lower()

In [ ]:
subtitles.head()

In [ ]:
subtitles.describe()

In [ ]:
subtitles[subtitles['data'].duplicated(keep=False)]

In [ ]:
subtitles = subtitles.drop(subtitles[subtitles['data'].duplicated()].index, axis=0)

## movie_labels

In [ ]:
movie_labels.isna().sum()

In [ ]:
movie_labels.describe()

### movie

In [ ]:
movie_labels['movie'] = movie_labels['movie'].str.lower()

In [ ]:
movie_labels.duplicated().sum()

In [ ]:
movie_labels = movie_labels.drop_duplicates()

In [ ]:
print(movie_labels[movie_labels['movie'].duplicated(keep=False)].index)
movie_labels[movie_labels['movie'].duplicated(keep=False)]

https://moviesbylevels.wordpress.com/

In [ ]:
movie_labels = movie_labels.drop([84, 99], axis=0)

### level

In [ ]:
movie_labels = movie_labels.reset_index(drop=True)

In [ ]:
movie_labels['level'].value_counts()

In [ ]:
indexes = movie_labels[movie_labels['level'] == 'A2/A2+, B1'].index
movie_labels[movie_labels['level'] == 'A2/A2+, B1']

In [ ]:
movie_labels['level'].iloc[indexes[0]] = 'A2'
movie_labels['level'].iloc[indexes[1]] = 'B1'
movie_labels['level'].iloc[indexes[2]] = 'B1'
movie_labels['level'].iloc[indexes[3]] = 'A2'

In [ ]:
indexes = movie_labels[movie_labels['level'] == 'B1, B2'].index
movie_labels[movie_labels['level'] == 'B1, B2']

In [ ]:
movie_labels['level'].iloc[indexes[0]] = 'B1'
movie_labels['level'].iloc[indexes[1]] = 'B1'
movie_labels['level'].iloc[indexes[2]] = 'B1'
movie_labels['level'].iloc[indexes[3]] = 'B2'
movie_labels['level'].iloc[indexes[4]] = 'B2'
movie_labels['level'].iloc[indexes[5]] = 'B2'
movie_labels['level'].iloc[indexes[6]] = 'B2'
movie_labels['level'].iloc[indexes[7]] = 'B1'

In [ ]:
movie_labels['level'] = movie_labels['level'].str.replace('A2/A2+', 'A2', regex=False)

In [ ]:
movie_labels['level'].value_counts()

## merge

In [ ]:
df = pd.merge(subtitles, movie_labels, how='outer', on=['movie'])

In [ ]:
display(df.info())
df.head()

In [ ]:
display(df.describe())
df.isna().sum()

### data

In [ ]:
df[df['data'].isna()]

#### its_a_wonderful_life(1946)

In [ ]:
df[df['movie'].str.contains('wonderful')]

In [ ]:
df['level_x'].iloc[207] = df['level_y'].iloc[276]
df = df.drop(276, axis=0)
df = df.reset_index(drop=True)

#### he secret life of pets.en

In [ ]:
df[df['movie'].str.contains('pet')]

https://subscene.com/subtitles/the-secret-life-of-pets/english/1440256

In [ ]:
subs = open('Missing_subtitles/The.Secret.Life.of.Pets.2016.720p.BRRip.x264.AAC-ETRG.srt', 'r', encoding='ISO-8859-1')
df['data'].iloc[274] = subs.read()
subs.close()

#### up (2009)

In [ ]:
df[df['movie'].str.contains('up')]

In [ ]:
df['level_x'].iloc[267] = df['level_y'].iloc[275]
df = df.drop(275, axis=0)
df = df.reset_index(drop=True)

#### glass onion

In [ ]:
df[df['movie'].str.contains('onion')]

https://subscene.com/subtitles/glass-onion-a-knives-out-mystery/english/2960299

In [ ]:
subs = open('Missing_subtitles/Glass.Onion.A Knives.Out.Mystery.2022.1080p.NF.WEB-DL.DDP5.1.Atmos.x264.srt', 'r',  encoding='ISO-8859-1')
df['data'].iloc[275] = subs.read()
subs.close()

#### matilda(2022)

In [ ]:
df[df['movie'].str.contains('matilda')]

https://subscene.com/subtitles/roald-dahls-matilda-the-musical/english/2968073

In [ ]:
subs = open("Missing_subtitles/Roald Dahl's Matilda the Musical (2022).srt", 'r',  encoding='ISO-8859-1')
df['data'].iloc[276] = subs.read()
subs.close()

#### bullet train

In [ ]:
df[df['movie'].str.contains('train')]

https://subscene.com/subtitles/bullet-train/english/3035799

In [ ]:
subs = open('Missing_subtitles/Bullet.Train.Eng.2022.NoHI.srt', 'r',  encoding='ISO-8859-1')
df['data'].iloc[277] = subs.read()
subs.close()

#### thor: love and thunder

In [ ]:
df[df['movie'].str.contains('thor')]

https://subscene.com/subtitles/thor-love-and-thunder/english/2878411

In [ ]:
subs = open('Missing_subtitles/Thor.Love.and.Thunder.2022.1080p.WEBRip.X264.DD5.1.srt', 'r',  encoding='ISO-8859-1')
df['data'].iloc[278] = subs.read()
subs.close()

#### lightyear

In [ ]:
df[df['movie'].str.contains('lightyear')]

https://subscene.com/subtitles/lightyear/english/2857387

In [ ]:
subs = open('Missing_subtitles/Lightyear.2022.WEBRip.x264-PS.srt', 'r',  encoding='ISO-8859-1')
df['data'].iloc[279] = subs.read()
subs.close()

#### the grinch

In [ ]:
df[df['movie'].str.contains('grinch')]

In [ ]:
df = df.drop(280, axis=0)
df = df.reset_index(drop=True)

#### Дубликаты

In [ ]:
df['data'].duplicated().sum()

### moive

In [ ]:
df['movie'].duplicated().sum()

### level

In [ ]:
df[df['level_x'].isna()]

In [ ]:
df['level_x'] = df['level_x'].fillna(df['level_y'])

In [ ]:
display(df['level_y'].unique())
df['level_x'].value_counts()

In [ ]:
indexes = df[(df['level_x'] == 'Unlabeled') & (df['level_y'].isna())].index
df[(df['level_x'] == 'Unlabeled') & (df['level_y'].isna())]

In [ ]:
df['level_x'].iloc[indexes[0]] = 'B2'
df['level_x'].iloc[indexes[1]] = 'B2'
df['level_x'].iloc[indexes[2]] = 'A2'
df['level_x'].iloc[indexes[3]] = 'A2' #A1 not present in dataset
df['level_x'].iloc[indexes[4]] = 'A2'
df['level_x'].iloc[indexes[5]] = 'B2'
df['level_x'].iloc[indexes[6]] = 'B2'
df['level_x'].iloc[indexes[7]] = 'B2'

In [ ]:
df.iloc[23][2]

In [ ]:
df['level_x'] = df[['level_x', 'level_y']].apply(lambda x: x['level_x'] if x['level_x'] != 'Unlabeled' else x['level_y'], axis=1)

In [ ]:
df['level_x'].unique()

In [ ]:
df = df.drop('level_y', axis=1)

In [ ]:
df = df.rename(columns={'level_x': 'level'})

In [ ]:
df.head()

## subtitle processing

In [ ]:
def parse_subtitles(data):
    subtitle_generator = srt.parse(data, ignore_errors=True);
    subs = list(subtitle_generator)
    subs_text = []
    for i in range(len(subs)):
        subs_text.append(subs[i].content)
    
    return(' '.join(subs_text))

In [ ]:
def remove_html_tags(text):
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [ ]:
def tokenize_and_clean(data):
    data_tokenized = nltk.word_tokenize(data)
    stopwords = nltk.corpus.stopwords.words('english')
    data_cleaned = [word for word in data_tokenized if word.lower() not in stopwords]
    data_cleaned = [word.lower() for word in data_cleaned if word.isalpha()]
    lemm = WordNetLemmatizer()
    data_lemmatized = [lemm.lemmatize(w) for w in data_cleaned]
    return data_lemmatized

In [ ]:
df['data'] = df['data'].apply(parse_subtitles)
df['data'] = df['data'].apply(remove_html_tags)

In [ ]:
df['data'] = df['data'].apply(tokenize_and_clean)

In [ ]:
level_a2 = df[df['level'] == 'A2']['data'].values
level_a2 = np.concatenate(level_a2, axis=0)
level_b1 = df[df['level'] == 'B1']['data'].values
level_b1 = np.concatenate(level_b1, axis=0)
level_b2 = df[df['level'] == 'B2']['data'].values
level_b2 = np.concatenate(level_b2, axis=0)
level_c1 = df[df['level'] == 'C1']['data'].values
level_c1 = np.concatenate(level_c1, axis=0)

# Исследовательский анализ данных

## level

In [ ]:
level_counts = df.groupby('level')['level'].count()
fig, ax = plt.subplots(figsize = (12,12))
ax.grid(False)
fig.patch.set_facecolor('w')
level_counts.plot(kind='pie', autopct='%1.1f%%',  textprops={'fontsize': 14})

## data

### A2

In [ ]:
plt.figure(figsize=(12, 20))
wordcloud = WordCloud(collocations=False, stopwords=STOPWORDS, background_color='black').generate(' '.join(level_a2))
plt.imshow(wordcloud)
plt.axis("off")
plt.title('A2')
plt.show()

### B1

In [ ]:
plt.figure(figsize=(12, 20))
wordcloud = WordCloud(collocations=False, stopwords=STOPWORDS, background_color='black').generate(' '.join(level_b1))
plt.imshow(wordcloud)
plt.axis("off")
plt.title('B1')
plt.show()

### B2

In [ ]:
plt.figure(figsize=(12, 20))
wordcloud = WordCloud(collocations=False, stopwords=STOPWORDS, background_color='black').generate(' '.join(level_b2))
plt.imshow(wordcloud)
plt.axis("off")
plt.title('B2')
plt.show()

### C1

In [ ]:
plt.figure(figsize=(12, 20))
wordcloud = WordCloud(collocations=False, stopwords=STOPWORDS, background_color='black').generate(' '.join(level_c1))
plt.imshow(wordcloud)
plt.axis("off")
plt.title('C1')
plt.show()

# Обучение моделей

## Pre-trained transformer

In [ ]:
df['data'] = df['data'].apply(' '.join)

In [ ]:
ds = Dataset.from_pandas(df.drop('movie', axis=1))
ds = ds.rename_column('level', 'label')
ds = ds.class_encode_column("label")

In [ ]:
ds

In [ ]:
model_nm = 'microsoft/deberta-v3-large'
tokz = AutoTokenizer.from_pretrained(model_nm, use_fast=True)

In [ ]:
def tokz_func(data):
    return tokz(data['data'])

In [ ]:
ds_tokenized = ds.map(tokz_func, batched=True, remove_columns='data')

In [ ]:
ds_split = ds_tokenized.train_test_split(test_size=0.2, stratify_by_column='label', seed=1234)

In [ ]:
def correlation(eval_pred):
    return {'pearson': np.corrcoef(*eval_pred)[0][1]}

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
bs = 128
args = TFTrainingArguments('outputs', learning_rate=8e-5, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True, evaluation_strategy="epoch",
                         per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2, num_train_epochs=4, weight_decay=0.01, report_to='none')
trainer = TFTrainer(model, args, train_dataset=ds_split['train'], eval_dataset=ds_split['test'], compute_metrics=correlation)

In [ ]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 
 # my output was => ['/device:CPU:0']
 # good output must be => ['/device:CPU:0', '/device:GPU:0']

In [ ]:
tf.debugging.set_log_device_placement(True) 

In [ ]:
tf.enable_eager_execution() 



# Explicitly place tensors on the DirectML device 

with tf.device('/DML:0'): 
    a = tf.constant([1.0, 2.0, 3.0]) 
    b = tf.constant([4.0, 5.0, 6.0]) 



c = tf.add(a, b) 

print(c)

In [ ]:
import sys
print (sys.version)
 # 3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
import tensorflow as tf
print(tf.__version__)
 # my output was => 1.13.1

In [ ]:
#trainer.train()